In [1]:
%%capture
!pip install --upgrade datasets
!pip install --upgrade evaluate
!pip install --upgrade transformers huggingface_hub


In [2]:
from datasets import load_from_disk, Dataset

# Load the datasets back
train = load_from_disk("/content/drive/MyDrive/Twi_ASR/2_processed_dataset/train_dataset")
test = load_from_disk("/content/drive/MyDrive/Twi_ASR/2_processed_dataset/test_dataset")

In [3]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/Twi_ASR/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [4]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [5]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [6]:
from transformers import Wav2Vec2ForCTC

model_checkpoint = "/content/drive/MyDrive/wav2vec2-large-xlsr-twi/checkpoint-2000"  # Replace with the path to your trained checkpoint
model = Wav2Vec2ForCTC.from_pretrained(model_checkpoint)


In [7]:
model.eval()

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


def log_predictions(batch):
    # Input values (audio features) and labels
    input_values = batch["input_values"][:2].to(device)
    labels = batch["labels"][:2] # Optional for comparison

    # Get logits (predictions) from the model
    with torch.no_grad():
        logits = model(input_values).logits

    # Convert logits to predicted token IDs
    pred_ids = torch.argmax(logits, axis=-1)

    # Decode predictions and ground truth
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(labels) if labels is not None else None

    # Print predictions and ground truth (if available)
    print("Predictions:", pred_str)
    if label_str:
        print("Ground Truth:", label_str)




NameError: name 'model' is not defined

In [ ]:
# Call log_predictions with a batch of data
log_predictions(train).to(device)
